In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
import calendar
import json
from scipy import stats
from scipy.stats import skew
from math import sqrt
from numpy import mean, var
from sklearn import preprocessing
from fastapi import FastAPI

In [16]:
#leemos los archivos

archivo1 = 'movies_dataset_modificado.csv'
archivo2 = 'credits_modificado.csv'
archivo3 = 'dataset_concatenado'

df_movies = pd.read_csv(archivo1)
df_credits = pd.read_csv(archivo2)

#concatenamos los archivos usando 'id' como clave
df_concatenado = pd.merge(df_movies, df_credits, on='id', how='left')

#guardamos el archivo concatenado
df_concatenado.to_csv(archivo3, index=False)

In [17]:
df = pd.read_csv('dataset_concatenado.csv')

#convertimos la columna 'release_date' a datetime
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')

In [18]:
'Se ingresa un mes en idioma Español. Debe devolver la cantidad de películas que fueron estrenadas en el mes consultado en la totalidad del dataset.'

#mapeamos los nombres de los meses en español a sus numeros correspondientes
def cantidad_filmaciones_mes(mes: str):
    meses_espanol = {
        'enero': 1, 'febrero': 2, 'marzo': 3, 'abril': 4,
        'mayo': 5, 'junio': 6, 'julio': 7, 'agosto': 8,
        'septiembre': 9, 'octubre': 10, 'noviembre': 11, 'diciembre': 12
    }
    mes_num = meses_espanol.get(mes.lower())

#manejamos errores
    if mes_num is None:
        return {"mensaje": f"Mes '{mes}' no válido. Debe ser un mes en español."}

#contamos las peliculas estrenadas en un mes específico
    cantidad = len(df[df['release_date'].dt.month == mes_num])
    return {"mensaje": f"{cantidad} cantidad de películas fueron estrenadas en el mes de {mes.capitalize()}"}

#llamamos a la función y mostramos el resultado
resultado = cantidad_filmaciones_mes("agosto")
print(resultado["mensaje"])

3951 cantidad de películas fueron estrenadas en el mes de Agosto


In [19]:
'Se ingresa un día en idioma Español. Debe devolver la cantidad de películas que fueron estrenadas en día consultado en la totalidad del dataset.'

#mapeamos los nombres de los dias en español a sus numeros correspondientes
def cantidad_filmaciones_dia(dia: str):
    dias_espanol = {
        'lunes': 0, 'martes': 1, 'miércoles': 2, 'jueves': 3,
        'viernes': 4, 'sábado': 5, 'domingo': 6
    }
    dia_num = dias_espanol.get(dia.lower())

#manejamos errores
    if dia_num is None:
        return {"mensaje": f"Día '{dia}' no válido. Debe ser un día de la semana en español."}

#contamos las peliculas estrenadas en un día en específico
    cantidad = len(df[df['release_date'].dt.dayofweek == dia_num])
    return {"mensaje": f"{cantidad} cantidad de películas fueron estrenadas en los días {dia.capitalize()}"}

#llamamos a la función y mostramos el resultado
resultado = cantidad_filmaciones_dia("martes")
print(resultado["mensaje"])

5769 cantidad de películas fueron estrenadas en los días Martes


In [23]:
'Se ingresa el título de una filmación esperando como respuesta el título, el año de estreno y el score.'

#definimos la función y buscamos la película especificada
def score_titulo(titulo_de_la_filmacion: str):
    titulo_busqueda = titulo_de_la_filmacion.lower().strip()
    pelicula = df[df['title'].str.lower().str.strip() == titulo_busqueda]
    
#manejamos los errores si no se encuentra la película
    if pelicula.empty:
        return {"mensaje": f"No se encontró la película '{titulo_de_la_filmacion}'."}

#extraemos cada uno de los datos solicitados (título, año y score)
    titulo = pelicula['title'].iloc[0]
    anio_estreno = pelicula['release_date'].dt.year.iloc[0]
    score = pelicula['popularity'].iloc[0]

    return {"mensaje": f"La película '{titulo}' fue estrenada en el año {anio_estreno} con un score de {score}."}

#llamamos a la función y mostramos el resultado
resultado = score_titulo("the avengers")
print(resultado["mensaje"])

La película 'The Avengers' fue estrenada en el año 1998 con un score de 9.562953.


In [24]:
'Se ingresa el título de una filmación esperando como respuesta el título, la cantidad de votos y el valor promedio de las votaciones. La misma variable deberá de contar con al menos 2000 valoraciones, caso contrario, debemos contar con un mensaje avisando que no cumple esta condición y que por ende, no se devuelve ningun valor.'

#definimos la función y buscamos la película especificada
def votos_titulo(titulo_de_la_filmacion: str):
    pelicula = df[df['title'].str.lower() == titulo_de_la_filmacion.lower()]

#manejamos los errores si no se encuentra la película
    if pelicula.empty:
        return {"mensaje": f"No se encontró la película '{titulo_de_la_filmacion}'."}
    
#extraemos cada uno de los datos solicitados (cantidad de votos y promedio de votos)
    cantidad_votos = pelicula['vote_count'].iloc[0]
    promedio_votos = pelicula['vote_average'].iloc[0]

#condicion de las 2000 valoraciones
    if cantidad_votos < 2000:
        return {"mensaje": f"La película '{pelicula['title'].iloc[0]}' no cumple con el mínimo de 2000 valoraciones."}

    return {"mensaje": f"La película '{pelicula['title'].iloc[0]}' fue estrenada en el año {pelicula['release_date'].dt.year.iloc[0]}. La misma cuenta con un total de {cantidad_votos} valoraciones, con un promedio de {promedio_votos}."}

#llamamos a la función y mostramos el resultado
resultado = votos_titulo("iron man")
print(resultado["mensaje"])

La película 'Iron Man' fue estrenada en el año 2008. La misma cuenta con un total de 8951.0 valoraciones, con un promedio de 7.4.


In [25]:
'Se ingresa el nombre de un actor que se encuentre dentro de un dataset debiendo devolver el éxito del mismo medido a través del retorno. Además, la cantidad de películas que en las que ha participado y el promedio de retorno. La definición no deberá considerar directores.'

def get_actor(nombre_actor: str):
 
    #filtramos las películas del actor
    peliculas_actor = df_credits[df_credits['name'].str.lower() == nombre_actor.lower()]

    #manejamos los errores
    if peliculas_actor.empty:
        return {"mensaje": f"No se encontró al actor '{nombre_actor}' o no tiene películas en el dataset."}

     #obtenemos los IDs de las películas del actor
    ids_peliculas_actor = peliculas_actor['id'].unique().tolist()

    #filtramos las películas
    peliculas_actor_movies = df[df['id'].isin(ids_peliculas_actor)]

    #calculamos el retorno promedio
    retorno_promedio = peliculas_actor_movies['return'].mean()

    return {
        "mensaje": f"El actor {nombre_actor.capitalize()} ha participado de {len(peliculas_actor_movies)} cantidad de filmaciones, el mismo ha conseguido un retorno de {peliculas_actor_movies['return'].sum()} con un promedio de {retorno_promedio:.2f} por filmación."
    }

#llamamos a la función y mostramos el resultado
resultado = get_actor("Robert Downey Jr.")
print(resultado["mensaje"])

El actor Robert downey jr. ha participado de 88 cantidad de filmaciones, el mismo ha conseguido un retorno de 0.0 con un promedio de 0.00 por filmación.


In [26]:
'Se ingresa el nombre de un director que se encuentre dentro de un dataset debiendo devolver el éxito del mismo medido a través del retorno. Además, deberá devolver el nombre de cada película con la fecha de lanzamiento, retorno individual, costo y ganancia de la misma.'

def get_director(nombre_director: str):
    #filtramos las películas del director
    peliculas_director = df[(df['job'] == 'Director') & (df['name'] == nombre_director)]

    #manejamos los errores
    if peliculas_director.empty:
        return {"mensaje": f"No se encontró al director '{nombre_director}' o no tiene películas en el dataset."}

    #obtenemos los IDs de las películas del director
    ids_peliculas_director = peliculas_director['id'].tolist()

    #filtramos las películas
    peliculas_director_movies = df[df['id'].isin(ids_peliculas_director)].copy()

    #convertimos la columna 'release_date' a datetime
    peliculas_director_movies['release_date'] = pd.to_datetime(peliculas_director_movies['release_date'], errors='coerce')

    #calculamos el retorno promedio
    retorno_promedio = peliculas_director_movies['return'].mean()

    #creamos la lista de películas
    lista_peliculas = []
    for index, row in peliculas_director_movies.iterrows():
        if pd.notna(row['release_date']):
            lista_peliculas.append({
                "titulo": row["title"],
                "fecha_lanzamiento": row["release_date"].strftime("%Y-%m-%d"),
                "retorno_individual": row["return"],
                "costo": row["budget"],
                "ganancia": row["revenue"]
            })
        else:
            lista_peliculas.append({
                "titulo": row["title"],
                "fecha_lanzamiento": None,
                "retorno_individual": row["return"],
                "costo": row["budget"],
                "ganancia": row["revenue"]
            })

    return {
        "mensaje": f"El director {nombre_director.capitalize()} ha dirigido {len(peliculas_director_movies)} cantidad de filmaciones, el mismo ha conseguido un retorno de {peliculas_director_movies['return'].sum()} con un promedio de {retorno_promedio:.2f} por filmación.",
        "peliculas": lista_peliculas
    }

# Llamar a la función y mostrar el resultado
resultado = get_director('Pedro Armendáriz Jr.')
print(resultado)

{'mensaje': 'El director Pedro armendáriz jr. ha dirigido 20 cantidad de filmaciones, el mismo ha conseguido un retorno de 0.0 con un promedio de 0.00 por filmación.', 'peliculas': [{'titulo': 'Knock on Wood', 'fecha_lanzamiento': '1981-03-01', 'retorno_individual': 0.0, 'costo': 1, 'ganancia': 0.0}, {'titulo': 'Knock on Wood', 'fecha_lanzamiento': '1981-03-01', 'retorno_individual': 0.0, 'costo': 1, 'ganancia': 0.0}, {'titulo': 'Knock on Wood', 'fecha_lanzamiento': '1981-03-01', 'retorno_individual': 0.0, 'costo': 1, 'ganancia': 0.0}, {'titulo': 'Knock on Wood', 'fecha_lanzamiento': '1981-03-01', 'retorno_individual': 0.0, 'costo': 1, 'ganancia': 0.0}, {'titulo': 'Knock on Wood', 'fecha_lanzamiento': '1981-03-01', 'retorno_individual': 0.0, 'costo': 1, 'ganancia': 0.0}, {'titulo': 'Knock on Wood', 'fecha_lanzamiento': '1981-03-01', 'retorno_individual': 0.0, 'costo': 1, 'ganancia': 0.0}, {'titulo': 'Knock on Wood', 'fecha_lanzamiento': '1981-03-01', 'retorno_individual': 0.0, 'costo':